In [ ]:
%config Completer.use_jedi=False

import sys
import os
import glob
import matplotlib.pyplot as plt
import numpy as np

from refsimu_singletrait import reload_simulation
from refsimu_singletrait import microbe_pop_on

In [ ]:
microbe_pop_on()

In [ ]:
def get_diag_collector(rep_dir):
    eco = reload_simulation(rep_dir,-1)
    print(eco.substrates.names)
    diagcollector = eco.get_diag_collector()
    tl_files = sorted(glob.glob('{}/timelines_dump_*.hdf5'.format(rep_dir)))
    dump_file = tl_files[0]
    diagcollector.load_from_dumpfile(dump_file)
    for k,d in diagcollector.timelines.items():
        print(k)
        print([kk for kk in d.keys()])
    return diagcollector



In [ ]:
def collect_diag_stats(dc_dict, dc_key, dc_comp, filter_func):
    da = np.array([filter_func(dc.timelines[dc_key][dc_comp]['values']) for k, dc in dc_dict.items()])
    times = np.array([dc.timelines[dc_key][dc_comp]['times'] for k, dc in dc_dict.items()])
    res = {
        'times': times[0,:],
        'min': np.min(da, axis=0),
        'mean': np.mean(da, axis=0),
        'max': np.max(da, axis=0),
        'std': np.std(da,axis=0),
    }
    return res

In [ ]:
base_dir = './MACHIN/' # replace by your own
# base_dir = './BIDULE/' # replace by your own
base_dir = './TESTSTOECH2/' # replace by your own

# base_dir = './MACHIN_TEST/'
sdirs = [f for f in glob.glob(os.path.join(base_dir,'*')) if os.path.isdir(f)]
dlists = {}
for sdir in sdirs:
    skey = '_'.join(os.path.basename(sdir).split('_')[1:3])
    dlist = glob.glob(os.path.join(sdir,'replica_*'))
    dlist = [d for d in dlist if os.path.isdir(d)]
    dlists[skey] = dlist

for k,d in dlists.items():
    print('*'*20,'\n',k)
    for i,r in enumerate(d):
        print(i,r)

In [ ]:
diags = {}
for k,d in dlists.items():
    tmp_d = {}
    for i,r in enumerate(d):
        tmp_d[i] = get_diag_collector(r)
    diags[k] = tmp_d

In [ ]:
# dc_dict = diags['CELLULOSEX1_Mut']
dc_key, dc_comp = 'space_sum','microbes'

def filter_func_reltmass(vals):
    dat = np.sum(vals,axis=-1)
    sdat = np.sum(dat,axis=1)
    dat = dat / sdat[:,np.newaxis]
    return dat

diag_stats = {}
for k,d in diags.items():
    diag_stats[k] = collect_diag_stats(d,dc_key,dc_comp, filter_func_reltmass)

In [ ]:
for k,d in diag_stats.items():
    fig, ax = plt.subplots(1,1,figsize=(14,4))
    labs = ['Tax {}'.format(i) for i in range(d['mean'].shape[1])]
    lc= ax.plot(d['times'],d['mean'],label=labs)
    cols = [l.get_color() for l in lc]
    ddown = d['mean']-d['std']
    dup = d['mean']+d['std']

    for itax, col in enumerate(cols):
        ax.fill_between(d['times'], ddown[:,itax],dup[:,itax],
                    color = col,
                    alpha=0.2
                       )
    ax.legend(ncol=3)
    ax.grid()
    ax.set_title(k)

In [ ]:
for k,d in diag_stats.items():
    fig, ax = plt.subplots(1,1,figsize=(14,4))
    labs = ['Tax {}'.format(i) for i in range(d['mean'].shape[1])]
    lc= ax.plot(d['times'],d['std'],label=labs)
    cols = [l.get_color() for l in lc]
    ax.legend(ncol=3)
    ax.grid()
    ax.set_title(k)

In [ ]:
sel_1 = 'CELLULOSEX1_NoMut'
for itax in range(10):
    fig, ax = plt.subplots(1,1, figsize=(12,4))
    for i,d in diags[sel_1].items():
        dat = np.sum(d.timelines['space_sum']['microbes']['values'][:,:,:],axis=-1)
        sdat = np.sum(dat,axis=1)
        dat = dat / sdat[:,np.newaxis]
        t = d.timelines['space_sum']['microbes']['times']
        ax.plot(t,dat[:,itax],label='rep {} tax {}'.format(i,itax))
    ax.legend(loc=2,ncol=2)
    ax.grid()
    ax.set_xlabel('time')
    ax.set_ylabel('Total mass relative');
    ax.set_title('Taxon {}'.format(itax))
    ax.set_ylim(0.05,0.1)
    

In [ ]:
sel_1 = 'CELLULOSEX1_Mut'
for itax in range(10):
    fig, ax = plt.subplots(1,1, figsize=(12,4))
    for i,d in diags[sel_1].items():
        dat = np.sum(d.timelines['space_sum']['microbes']['values'][:,:,:],axis=-1)
        sdat = np.sum(dat,axis=1)
        dat = dat / sdat[:,np.newaxis]
        t = d.timelines['space_sum']['microbes']['times']
        ax.plot(t,dat[:,itax],label='rep {} tax {}'.format(i,itax))
    ax.legend(loc=2,ncol=2)
    ax.grid()
    ax.set_xlabel('time')
    ax.set_ylabel('Total mass relative');
    ax.set_title('Taxon {}'.format(itax))
    ax.set_ylim(0,0.2)
    

In [ ]:
diag_stats2 = {}
diag_stats3 = {}
diag_stats4 = {}
def get_atom_sum(vals):
    return np.sum(vals,axis=-1)

def get_decay_rate(vals):

    dsum = np.sum(vals,axis=-1)
#     dsum[dsum > 0] = 0
    dgrad = np.gradient(dsum)
    print(dgrad.shape)
    dgrad[dgrad > 0] = 0
#     conv = np.convolve(dgrad,kern,'same')
    return dgrad

for k,d in diags.items():
    print(k)
    diag_stats2[k] = collect_diag_stats(d,'space_comp_sum','substrates', filter_func=get_atom_sum)
    diag_stats3[k] = collect_diag_stats(d,'space_comp_sum','substrates', filter_func= get_decay_rate)
    diag_stats4[k] = collect_diag_stats(d,'space_sum','substrates',filter_func=get_atom_sum)

In [ ]:
fig, ax  = plt.subplots(1,1,figsize=(14,4))
for k,d in diag_stats2.items():
    ax.plot(d['times'],d['mean'],label=k)
    ax.set_title('Substrate total mass')
    ax.set_xlabel('time')
    ax.set_ylabel('Rate')
    ax.legend()
    ax.grid('on')

In [ ]:
wsize = 365
kern = np.ones((wsize))
kern = kern / np.sum(kern)


fig, ax  = plt.subplots(1,1,figsize=(14,4))
for k,d in diag_stats3.items():
    ax.plot(d['times'],d['mean'],label=k)
    ax.set_title('Substrate decay rate')
    ax.set_xlabel('time')
    ax.set_ylabel('Rate')
    ax.legend()
    ax.grid('on')

    
fig, ax  = plt.subplots(1,1,figsize=(14,4))
for k,d in diag_stats3.items():
    conv = np.convolve(d['mean'],kern,'valid')
    tconv = np.convolve(d['times'],kern,'valid')
    ax.plot(tconv,conv,label=k)
    ax.set_title('Substrate decay rate (moving average {})'.format(wsize))
    ax.set_xlabel('time')
    ax.set_ylabel('Rate')
    ax.legend()
    ax.grid('on')


In [ ]:
for isub in range(12):
    fig, ax  = plt.subplots(1,1,figsize=(14,4))
    for k,d in diag_stats4.items():
        ax.plot(d['times'],d['mean'][:,isub],label=k)
        ax.set_xlabel('time')
        ax.set_ylabel('Substrate {}'.format(isub))
        ax.grid('on')
        ax.legend()